# Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

In [2]:
from mlp import NeuralNetwork

# Load Data

Let's compare the implementation on MNIST data

In [3]:
# Load and preprocess MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [4]:
mnist_train = datasets.MNIST('./data', train=True, download=True, transform=transform)
mnist_test = datasets.MNIST('./data', train=False, transform=transform)

In [5]:
# Prepare data for NumPy implementation
X_train = mnist_train.data.numpy().reshape(-1, 784) / 255.0
y_train = mnist_train.targets.numpy()

X_test = mnist_test.data.numpy().reshape(-1, 784) / 255.0
y_test = mnist_test.targets.numpy()

# Train Models

## From Scratch Model

In [7]:
%%time
# Training my implementation
np_model = NeuralNetwork([784, 128, 64, 10])
np_model.train(X_train, y_train, epochs=5, batch_size=32, learning_rate=0.1)

Epoch 1/5, Accuracy: 0.9630
Epoch 2/5, Accuracy: 0.9737
Epoch 3/5, Accuracy: 0.9834
Epoch 4/5, Accuracy: 0.9853
Epoch 5/5, Accuracy: 0.9895
CPU times: user 1min 21s, sys: 2.47 s, total: 1min 24s
Wall time: 11.5 s


## PyTorch

In [8]:
class PyTorchNN(nn.Module):
    def __init__(self, layers):
        super(PyTorchNN, self).__init__()
        self.layers = nn.ModuleList()

        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i + 1]))
            if i < len(layers) - 2:
                self.layers.append(nn.ReLU())

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [9]:
%%time
# create the model with same layers
torch_model = PyTorchNN([784, 128, 64, 10])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(torch_model.parameters(), lr=0.1)

train_loader = DataLoader(
    TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train)),
    batch_size=32,
    shuffle=True,
)

for epoch in range(5):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = torch_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate accuracy
    with torch.no_grad():
        outputs = torch_model(torch.FloatTensor(X_test))
        _, predicted = torch.max(outputs.data, 1)
        accuracy = (predicted == torch.LongTensor(y_test)).sum().item() / len(y_test)
        print(f"Epoch {epoch+1}/5, Accuracy: {accuracy:.4f}")

Epoch 1/5, Accuracy: 0.9531
Epoch 2/5, Accuracy: 0.9677
Epoch 3/5, Accuracy: 0.9710
Epoch 4/5, Accuracy: 0.9743
Epoch 5/5, Accuracy: 0.9751
CPU times: user 12.5 s, sys: 1.25 s, total: 13.7 s
Wall time: 3.63 s
